## Select the analysis mode from the dropdown menu below.

In [1]:
breadboard_repo_path = r'C:\Users\Alex\Dropbox (MIT)\lab side projects\control software\breadboard-python-client\\'
import ipywidgets as widgets
from ipywidgets import interact
from measurement_directory import *
import sys
sys.path.insert(0, breadboard_repo_path)
from breadboard import BreadboardClient
bc = BreadboardClient(config_path='API_CONFIG.json') # enter your path to the API_config

def set_analysis_type(analysis_mode):
    global analysis_type
    analysis_type = analysis_mode
    print('analysis mode is {a_type} \n'.format(a_type = analysis_type))

mode_select_widget = interact(set_analysis_type,analysis_mode=['fake analysis' , 'fake analysis 2', 'ycam', 'zcam_dual_imaging'])

interactive(children=(Dropdown(description='analysis_mode', options=('fake analysis', 'fake analysis 2', 'ycam…

## Enter the run from today you wish to analyze.

In [2]:
print('today\'s measurement(s):')
print(todays_measurements())
watchfolder = measurement_directory()

today's measurement(s):
['foo', 'misplacedimages111701']
Enter name for this set of runs:foo


## Run the cell below to begin analyzing.

Check the output; it will update as images are analyzed.

To stop the analysis, press the interrupt kernel/stop button at the very top. You can restart the analysis by simply running the cell below again. The console output will be cleared, but not to worry, it's all in the log file.

In [3]:
print('loading matlab engine...')
import matlab.engine
eng = matlab.engine.start_matlab()
print('matlab engine loaded')

loading matlab engine...
matlab engine loaded


In [12]:
import os
import time
import datetime
import shutil
import warnings
import enrico_bot
from image_watchdog import getFileList

# import pandas as pd
# from numpy import isnan
# analysis_type = 'fake analysis'

# define analysis_function which accepts image filepath and returns dictionary of {analyzed_var_name:analysis_value}
if analysis_type == 'fake analysis':
    from random import randint
    def analysis_function(filepath):
        time.sleep(randint(0,2))
        return {'fake analysis 1': randint(0,42), 'fake analysis 2': randint(0,42)}
    analyzed_var_names = ['fake analysis 1', 'fake analysis 2']
elif analysis_type == 'fake analysis 2':
    from random import randint
    def analysis_function(filepath):
        time.sleep(randint(0,5))
        return {'fake analysis 3': randint(0,42), 'fake analysis 4': randint(0,42)}
    analyzed_var_names = ['fake analysis 3', 'fake analysis 4']
elif analysis_type == 'ycam':
    analyzed_var_names = ['bareNcntAverageMarqueeBoxValues', 
                          'COMX', 'COMY'] #TODO set MATLAB analyzed_var_names properly
    from matlab_wrapper import getMATLABanalysis
    def analysis_function(filepath, previous_settings = None):
        if previous_settings is None:
            matlab_dict = getMATLABanalysis(eng, filepath)
        else:
#             print('previous settings: ' + previous_settings)
#             return {'fakematlab1': randint(0,42), 'fakematlab2': randint(0,42)}, 'foo1'
            matlab_dict = getMATLABanalysis(eng, filepath, marqueeBox = previous_settings['marqueeBox'],
                                           normBox = previous_settings['normBox'])
        analysis_dict, settings = matlab_dict['analysis'], matlab_dict['settings']
        return analysis_dict, settings
elif analysis_type == 'zcam_dual_imaging':
    analyzed_var_names = ['K_bareNcntAverageMarqueeBoxValues', 'Na_bareNcntAverageMarqueeBoxValues',
                          'Na_COMX', 'Na_COMY'] #TODO set MATLAB analyzed_var_names properly
    from dual_imaging_wrapper import getDualImagingAnalysis
    def analysis_function(filepath, previous_settings = None):
        if previous_settings is None:
            matlab_dict = getDualImagingAnalysis(eng, filepath)
#         else:
#             matlab_dict = getMATLABanalysis(eng, filepath, marqueeBox = previous_settings['marqueeBox'],
#                                            normBox = previous_settings['normBox'])
        analysis_dict = matlab_dict['analysis']
        settings = None
        return analysis_dict, settings 

warnings.filterwarnings(
    "ignore", "Your application has authenticated using end user credentials")
warnings.filterwarnings(
    "ignore", "Could not find appropriate MS Visual C Runtime")
bc = BreadboardClient(config_path='API_CONFIG_fermi1.json')

def analyze_image(image_filename, previous_settings = None, output_previous_settings = True):
    #TODO adapt for triple imaging later
    abs_image_path = os.path.join(os.path.join(os.getcwd(), watchfolder), image_filename)
    settings = None
    print('{file} analyzing: '.format(file=image_filename))
    if analysis_type == 'fake analysis' or analysis_type == 'fake analysis 2':
        analysis_dict = analysis_function(image_filename)
    elif (analysis_type == 'ycam' or analysis_type == 'zcam_dual_imaging') and previous_settings is None:
        analysis_dict, settings = analysis_function(abs_image_path)#no marquee box yet
    elif (analysis_type == 'ycam' or analysis_type == 'zcam_dual_imaging'):
        analysis_dict, settings = analysis_function(abs_image_path, previous_settings)
    if not output_previous_settings:
        settings = None #forces user to select new marquee box for each shot
    for key in analyzed_var_names:
        print(key, analysis_dict[key])
    return analysis_dict, settings

def main():
    # Global settings
    refresh_time = 1  # seconds
    previous_settings = None
    print("\n\n Watching this folder for changes: " + watchfolder + "\n\n")
    unanalyzed_files = []
    done_files = []
    append_mode = True

    # Main Loop
    while True:
        if not os.path.exists(watchfolder):
            time.sleep(refresh_time)
            continue
        else:
            files, _ = getFileList(watchfolder)
            fresh_files = sorted(list(set(files).difference(set(done_files)).difference(set(unanalyzed_files))))
            unanalyzed_files += fresh_files #push newest files to top of stack
        
        for file in reversed(unanalyzed_files): #start from top of stack
            run_id = run_id_from_filename(file)
            if append_mode:
                run_dict = bc._send_message('get', '/runs/' + str(run_id) + '/').json()
                if set(analyzed_var_names).issubset(set(run_dict.keys())):
                    popped_file = [unanalyzed_files.pop()]
                    done_files += popped_file
                    continue
            analysis_dict, previous_settings = analyze_image(file, previous_settings)
#             resp = bc.append_analysis_to_run(run_id, analysis_dict)
            popped_file = [unanalyzed_files.pop()]
            done_files += popped_file
            print('\n')

        time.sleep(refresh_time)
# try:
#     main()
# except:
#     enrico_bot.post_message('analysis crashed, restart it.')
#     pass

main()



 Watching this folder for changes: 08\200811\foo


9_0.csv analyzing: 
fake analysis 1 22
fake analysis 2 9


8_0.csv analyzing: 
fake analysis 1 0
fake analysis 2 28


7_0.csv analyzing: 
fake analysis 1 17
fake analysis 2 15


6_0.csv analyzing: 
fake analysis 1 17
fake analysis 2 9


5_0.csv analyzing: 
fake analysis 1 2
fake analysis 2 14


4_0.csv analyzing: 
fake analysis 1 37
fake analysis 2 5


3_0.csv analyzing: 
fake analysis 1 23
fake analysis 2 24


2_0.csv analyzing: 
fake analysis 1 0
fake analysis 2 5


1_0.csv analyzing: 
fake analysis 1 9
fake analysis 2 42


16_0.csv analyzing: 


KeyboardInterrupt: 

In [ ]:

# def analysis_to_imagelog(image_log_df, idx, outputlog_filename, previous_settings = None,
#                         output_previous_settings = True):
#     image_filename = image_log_df.loc[idx, 'filename0']
#     abs_image_path = os.path.join(os.path.join(os.getcwd(), measurement_dir), image_filename)
#     print(abs_image_path)
#     settings = None
#     if analysis_type == 'fake analysis' or analysis_type == 'fake analysis 2':
#         analysis_dict = analysis_function(image_filename)
#     elif (analysis_type == 'ycam' or analysis_type == 'zcam_dual_imaging') and previous_settings is None:
#         analysis_dict, settings = analysis_function(abs_image_path)#no marquee box yet
#     elif (analysis_type == 'ycam' or analysis_type == 'zcam_dual_imaging'):
#         analysis_dict, settings = analysis_function(abs_image_path, previous_settings)
#     if not output_previous_settings:
#         settings = None #forces user to select new marquee box for each shot
#     image_log_df = pd.read_csv(outputlog_filename) #get updated copy of log to prevent overwriting
#     print('{file} analyzing: '.format(file=image_filename))
#     for key in analyzed_var_names:
#         print(key, analysis_dict[key])
#         image_log_df.loc[idx,key] = analysis_dict[key]
#     image_log_df.to_csv(outputlog_filename, index = False)
# #     print('{file} analyzed: '.format(file=image_filename) + str(analysis_dict))
#     return image_log_df, settings

#         for var_name in analyzed_var_names:
#             if var_name not in image_log_df.columns: #start new analysis type
# #             print('{analysis} analysis started'.format(analysis = analysis_type))
#                 image_log_df, previous_settings = analysis_to_imagelog(image_log_df, 0, watchfile, previous_settings)
#                 if 'analyzed_variables' not in image_log_df.columns:
#                     image_log_df.loc[0,'analyzed_variables'] = str(analyzed_var_names)
#                     image_log_df.to_csv(watchfile, index = False)
#                 else:
#                     old_analyzed_variables = set(translate_stringy_list(image_log_df.loc[0,'analyzed_variables']))
#                     updated_analyzed_variables = list(old_analyzed_variables.union(set(analyzed_var_names)))
#                     image_log_df.loc[0,'analyzed_variables'] = str(updated_analyzed_variables)
#                     image_log_df.to_csv(watchfile, index = False)
#             else: #queue un-analyzed images
#                 for idx in reversed(range(len(image_log_df.index))): #start from most recent image first
#                     if isnan(image_log_df.loc[idx, var_name]):
#                         image_log_df, previous_settings = analysis_to_imagelog(image_log_df, idx, watchfile, previous_settings)
#                         if isinstance(image_log_df.loc[idx,'analyzed_variables'],float):
#                             image_log_df.loc[idx,'analyzed_variables'] = str(analyzed_var_names)
#                             image_log_df.to_csv(watchfile, index = False)
#                         else:
#                             old_analyzed_variables = set(translate_stringy_list(image_log_df.loc[idx,'analyzed_variables']))
#                             updated_analyzed_variables = list(old_analyzed_variables.union(set(analyzed_var_names)))
#                             image_log_df.loc[idx,'analyzed_variables'] = str(updated_analyzed_variables)
#                             image_log_df.to_csv(watchfile, index = False)

            # # Write to Breadboard
            # try:
            #     resp = bc.post_images(
            #                 image_names = os.path.splitext(output_filename)[0],
            #                 filepath = output_filepath,
            #                 image_times = [dt],
            #                 auto_time = False # Add more information here
            #                 )
            #     if resp.status_code!=200:
            #         print(resp.text)
            # except: pass
            # names_old = names
#             length_old = len(image_log_df.index)

        # Wait before checking again